In [70]:
from dotenv import load_dotenv
from datetime import datetime, timezone
import os

from data_chunking import datachunk
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint as HuggingFaceHub
from langchain_community.vectorstores.pinecone import Pinecone
from pinecone import Pinecone as pc


class chatbot:
    def __init__(self, temperature=0.8, prompt_sourcedata=None, prompt_conv=None, user_name=None, session_id=None):
        load_dotenv()
        self.embeddings = HuggingFaceEmbeddings()
        self.index_name = "eer-transcripts-pdfs"
        self.pinecone_instance = pc(api_key=os.getenv('PINECONE_API_KEY'), embeddings=self.embeddings)
        
        # Self-assign parameters
        self.user_name = user_name
        self.session_id = session_id
        
        self.temperature = temperature
        
        # Instantiate the LLM
        self.llm = HuggingFaceHub(
            repo_id=os.getenv('repo_id'),
            temperature=temperature,
            top_p=0.8,
            top_k=50,
            huggingfacehub_api_token=os.getenv('HUGGINGFACE_API_KEY')
        )

    # Renamed method
    def default_prompt_sourcedata(self, chat_history, original_data, user_input, user_name):
        return f"""You are a chatbot assistant, working for the Experimenting Experiencing Reflecting (EER) Project, a research endeavor investigating the connections between art and science.
    You have access to a collection of documents, including descriptions of research activities, meeting transcripts, and other relevant materials.Your main task is to help the user explore and reflect on the EER project.     When possible, please cite source documents in your answer (calling the documents the transcript date, e.g. "2021-05-28", or website page after the "/". 
    You are now assisting the user "{user_name}" with their query: "{user_input}". Below is the relevant data from the EER group that was retrieved from the database for this query: "{original_data}". Based on this data, provide a concise answer to the user’s question in 1-3 sentences. The previous chat history for this session so far is: {chat_history} Your response:"""

    def default_prompt_conv(self, chat_history, user_input, llm_response, past_chat, user_name):
        return f"""You are an assistant observing conversations between the user and another LLM regarding meeting transcripts and content from the Experimenting Experiencing Reflecting (EER) Project, a research group investigating the connections between art and science.
    All interactions between people and the LLM are recorded and stored in your database. When people ask questions about the data, you get the question and the answer from the LLM. You use that data to search your database of past conversations for conversations that might be related. You will create a summary of those past conversations no longer than 4 sentences. Your summary should mention the name of the person involved in the past conversations, so that if the user wants to, they can follow up with them.
    Here is the last question asked by the user in this session: "{user_name}" asked: {user_input}
    Here is what the LLM you are watching responded with: “{llm_response}”
    Here is relevant data from past conversations that is relevant: {past_chat}
    Here is the chat history for this session, so that your response can be aware of the context: {chat_history}
    Your response: """

   # Method to retrieve documents from Pinecone index while excluding a specific session_id
    def retrieve_docs(self, input, index, excluded_session_id=None, k=5):
        # Retrieve past conversation data
        docsearch = Pinecone.from_existing_index(index, self.embeddings)
        
        if index == "eer-interaction-data":
            # Add metadata filter to exclude the given session_id
            search_kwargs = {
                "k": k,
                "filter": {
                    "session_id": {"$ne": excluded_session_id}
                }
            }
        else:
            search_kwargs = {
                "k": k
            }
            
        retriever = docsearch.as_retriever(search_kwargs=search_kwargs)
        docs = retriever.invoke(input)
        
        return docs

    # Method to generate response from LLM
    def get_llm_response(self, prompt):
        try:
            response = self.llm.invoke(prompt)
            return response
        except Exception as e:
            error = f"Error invoking LLM: {e}"
            return error
    
    # Method to format the context
    def format_context(self, documents, chat=False):
        context = ""
        
        for idx, doc in enumerate(documents, start=1):
            metadata = doc.metadata
            if chat:
                context += (
                    f'User {idx}: {metadata.get("user_name", "Unknown User")}\n'
                    f'Chat session {idx}: {metadata.get("session_id", "Unknown Session ID")}\n'
                    f'User Question: "{metadata.get("user_question", "Unknown Question")}"\n'
                    f'AI Response: "{metadata.get("ai_output", "Unknown Response")}"\n'
                    f"Date: {metadata.get('date', 'Unknown Date')}\n\n"
                )
            else:
                if metadata.get("page") is not None:
                    context += (
                        f"Document type: PDF {idx}\n"
                        f"Page content: {doc.page_content}\n"
                        f"Source: {metadata['source']}\n"
                        f"Page: {metadata['page']}\n"
                    )
                else:
                    context += (
                        f"Document type: Meeting Transcript Exerpt {idx}\n"
                        f"Person {idx}: {metadata.get('speaker_name', 'Unknown Speaker')}\n"
                        f"Date: {metadata.get('date_time', 'Unknown Date')}\n"
                        f"Content: {doc.page_content}\n\n"
                    )

        return context

    # Method to upsert data to Pinecone index
    def upsert_vectorstore(self, user_input, ai_output, user_name, user_location, session_id):
        # Pinecone index for chat data
        pinecone_instance_chat = pc(api_key=os.getenv('PINECONE_API_KEY'), embeddings=self.embeddings)
        index_name = "eer-interaction-data"
        environment = "gcp-starter"
        
        index = pinecone_instance_chat.Index(index_name, environment=environment)
                
        embedding = self.embeddings.embed_documents([user_input + ai_output])[0]
        
        # Upsert the summary embedding to the Pinecone index with metadata, including timestamp
        index.upsert(vectors=[
            {
                'id': session_id,
                'values': embedding,
                'metadata': {
                    "user_question": user_input,
                    "ai_output": ai_output,
                    "user_name": user_name,
                    "session_id": session_id,
                    "date": datetime.now(timezone.utc).isoformat(),
                    "user_location": user_location, 
                    "text": f"User input: {user_input}, AI output: {ai_output}"
                }
            }
        ])

    def pipeline(self, user_input, user_name, session_id, user_location, chat_history=None):
        # Step 0: Add chat history to the context
        if chat_history:
            chat_history = chat_history + "\n\n"
        else:
            chat_history = ""
        
        # Step 1: Retrieve source data
        source_data = self.retrieve_docs(user_input, "eer-transcripts-pdfs")
        formatted_source_data = self.format_context(source_data)

        # Step 2: Generate LLM response from source data
        sourcedata_response = self.get_llm_response(self.default_prompt_sourcedata(chat_history=chat_history, original_data = formatted_source_data, user_input = user_input, user_name=user_name))

        # Step 3: Retrieve past chat context
        past_chat_context = self.retrieve_docs(sourcedata_response, "eer-interaction-data", session_id)
        formatted_chat_context = self.format_context(past_chat_context, chat=True)
        
        # Step 5: Generate LLM response for conversation context, now considering combined chat history
        conversation_response = self.get_llm_response(self.default_prompt_conv(chat_history=chat_history, user_input=user_input, llm_response=sourcedata_response, past_chat=formatted_chat_context, user_name=user_name))

        # Step 6: Combine the responses
        ai_output = f"{sourcedata_response}\n\n{conversation_response}"
        
        print("AI Output: ", ai_output)
        print("sourcedaa_response: ", sourcedata_response)
        print("conversation_response: ", conversation_response)

        # Upsert to vector store
        self.upsert_vectorstore(user_input, ai_output, user_name, user_location, session_id)

        # Return a dictionary containing all relevant information
        return {
            "ai_output": ai_output,
            "source_data": source_data,
            "past_chat_context": past_chat_context
        }

In [71]:
cb = chatbot()

c:\Users\szh\Desktop\EER\EER-chatbot-UI\.venv\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\szh\.cache\huggingface\token
Login successful


In [73]:
cb.get_llm_response("test")

'Error invoking LLM: Model not loaded on the server: https://v2z2wd97mmpdh4nx.us-east-1.aws.endpoints.huggingface.cloud. Please retry with a higher timeout (current: 120).'

In [25]:
docs = cb.retrieve_docs("What is the EER project?", "eer-transcripts-pdfs")

In [72]:
cb.pipeline("What is the EER project?", "John Doe", "1234", "New York, NY", chat_history="")

AI Output:  Error invoking LLM: Model not loaded on the server: https://v2z2wd97mmpdh4nx.us-east-1.aws.endpoints.huggingface.cloud. Please retry with a higher timeout (current: 120).

Error invoking LLM: Model not loaded on the server: https://v2z2wd97mmpdh4nx.us-east-1.aws.endpoints.huggingface.cloud. Please retry with a higher timeout (current: 120).
sourcedaa_response:  Error invoking LLM: Model not loaded on the server: https://v2z2wd97mmpdh4nx.us-east-1.aws.endpoints.huggingface.cloud. Please retry with a higher timeout (current: 120).
conversation_response:  Error invoking LLM: Model not loaded on the server: https://v2z2wd97mmpdh4nx.us-east-1.aws.endpoints.huggingface.cloud. Please retry with a higher timeout (current: 120).


{'ai_output': 'Error invoking LLM: Model not loaded on the server: https://v2z2wd97mmpdh4nx.us-east-1.aws.endpoints.huggingface.cloud. Please retry with a higher timeout (current: 120).\n\nError invoking LLM: Model not loaded on the server: https://v2z2wd97mmpdh4nx.us-east-1.aws.endpoints.huggingface.cloud. Please retry with a higher timeout (current: 120).',
 'source_data': [Document(metadata={'date_time': 'Tue Jan 30 00:04:05 2024 ', 'source': '2024-01-30 09.03.37_rf.csv', 'speaker_name': 'Joseph Dumit'}, page_content='Can you ask it? What kind of aesthetics does EER approach compared with other approaches to aesthetics?'),
  Document(metadata={'page': 1.0, 'source': "..\\data\\EER-site-pages-pdf\\EER 'The Journeys and Territories of EER' – A Two-Day Workshop.pdf"}, page_content="Carte de Tendre, we mapped our EER projects and theirCopenhagen, DKBack'The Journeys and Territories ofEER' – A Two-Day WorkshopEER | 'The Journeys and Territories of EER' – A Two-Day Workshophttps://www.eer

In [1]:
import os
from typing import Optional, List, Dict, Any
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms.huggingface_endpoint import HuggingFaceEndpoint as HuggingFaceHub
from langchain_community.vectorstores import Pinecone
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser, Document
from langchain.retrievers import MultiQueryRetriever
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.runnables import RunnableLambda, RunnableSequence
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain_core.callbacks.base import BaseCallbackManager
from langchain.retrievers.multi_query import LineListOutputParser
from pinecone import Pinecone as pc
from uuid import uuid4
from dotenv import load_dotenv

load_dotenv()

C:\Users\szh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

: 

## attempt

In [98]:
class ChatBot:
    def __init__(self, custom_template: Optional[str] = None, repo_id: Optional[str] = None, temperature: float = 0.8):
        self.embeddings = HuggingFaceEmbeddings()
        self.index_name = "boteer"
        
        pinecone_instance = pc(api_key=os.getenv('PINECONE_API_KEY'), embeddings=self.embeddings)        
        self.docsearch = Pinecone.from_existing_index(self.index_name, self.embeddings)

        self.template = custom_template if custom_template else self.default_template()
        self.temperature = temperature
        self.repo_id = repo_id
        
        self.llm = HuggingFaceHub(
            repo_id=self.repo_id,
            temperature=temperature,
            top_p=0.8,
            top_k=50,
            huggingfacehub_api_token=os.getenv('HUGGINGFACE_API_KEY')
        )

        # Create the combine_docs_chain
        prompt = PromptTemplate(template=self.template + self.template_end(), input_variables=["context", "question"])
        self.combine_docs_chain = create_stuff_documents_chain(self.llm, prompt)

        # Create the retriever
        self.retriever = MultiQueryRetriever.from_llm(
            retriever=self.docsearch.as_retriever(),
            llm=self.llm
        )

         # Create the retrieval chain
        self.rag_chain = self.create_retrieval_chain()

        print("Chain assembled...")

    def default_template(self) -> str:
        return """
        You are a chatbot working for the Experimenting Experiencing Reflecting (EER) Project, a research endeavor investigating the connections between art and science.
        You have access to a collection of documents, including descriptions of research activities, meeting transcripts, and other relevant materials.
        All questions should pertain to the EER Project unless specified otherwise.
        When possible, please cite source documents in your answer (calling the documents the transcript date, e.g. "2021-05-28", or website page after the "/").
        """
    
    def template_end(self) -> str:
        return """
        Context: {context}
        Question: {question}
        Answer: 
        """
        
    def create_retrieval_chain(self):
        # Create the retrieval chain properly using the provided utility
        return create_retrieval_chain(
            retriever=self.retriever,
            combine_documents_chain=self.combine_docs_chain
        )
        
    def chat(self, question: str) -> str:
        # Format the input to match the expected input variables
        input_variables = {"input": question}
        response = self.rag_chain.invoke(input_variables)
        return response.get("answer", "I'm sorry, I couldn't generate an answer.")


In [ ]:
"""# Create a minimal callback manager
class DummyCallbackManager(CallbackManagerForRetrieverRun):
    def __init__(self):
        super().__init__(run_id="dummy_run_id", handlers=[], inheritable_handlers=[])
    def get_child(self):
        return self

run_manager = DummyCallbackManager()"""

## Trying

In [140]:
# Initialize HuggingFace Embeddings
embeddings = HuggingFaceEmbeddings()

# Initialize Pinecone
pinecone_instance = pc(api_key=os.getenv('PINECONE_API_KEY'), embeddings=embeddings)

# Connect to an existing Pinecone index
index_name = "boteer"
docsearch = Pinecone.from_existing_index(index_name, embeddings)

In [141]:
# Initialize the language model from HuggingFace
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    temperature=0.8,
    huggingfacehub_api_token=os.getenv('HUGGINGFACE_API_KEY')
)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\szh\.cache\huggingface\token
Login successful


In [131]:
# Create a MultiQueryRetriever
prompt = PromptTemplate(input_variables=["question"], template="Generate multiple search queries for the following question: {question}")
output_parser = LineListOutputParser()
llm_chain = LLMChain(llm=llm, prompt=prompt, output_parser=output_parser)

retriever = MultiQueryRetriever(
    retriever=docsearch.as_retriever(),
    llm_chain=llm_chain,
    include_original=True
)


In [143]:
# Define default templates
def default_template() -> str:
    return """
    You are a chatbot working for the Experimenting Experiencing Reflecting (EER) Project, a research endeavor investigating the connections between art and science.
    You have access to a collection of documents, including descriptions of research activities, meeting transcripts, and other relevant materials.
    All questions should pertain to the EER Project unless specified otherwise.
    """

def template_end() -> str:
    return """
    Context: {context}
    Question: {question}
    Answer: 
    """

In [144]:
# Create a minimal callback manager
class DummyCallbackManager(BaseCallbackManager):
    def get_child(self):
        return self

callback_manager = DummyCallbackManager(handlers=[], inheritable_handlers=[])

In [173]:
retriever.batch(query, run_manager=callback_manager)

[[Document(page_content='existence of AIS.', metadata={'date_time': 'Thu Mar 16 00:03:20 2023 ', 'source': '2023-03-16 09.01.59_rf.csv', 'speaker_name': 'Peter Dalsgaard'}),
  Document(page_content="this? Incidentally, I'm at an AI workshop AI and humanities workshop organized by Mads Rosendal all of Monday. So as I don't remember the schedule, but if I can join the date, I will great Yeah. Please", metadata={'date_time': 'Fri Dec 16 00:21:51 2022 ', 'source': '2022-12-16 08.40.00_rf.csv', 'speaker_name': 'Peter Dalsgaard'}),
  Document(page_content='a pretty well known thing. But what what I think is also really interesting here is that in many cases, we also see people, and maybe this is particular, or particularly salient for for AI is that people seem to ascribe meaning and agency to the machine. I mean, we even talk about our computer behaving in a stupid way. And we get angry at it as if it was a petulant child or something like that. And I think that maybe there could be somethi

In [172]:
# Define a query
query = "Tell me about the EER project."

# Retrieve documents
results = retriever.batch(query, run_manager=callback_manager)

# Combine documents content into a single context
context = "\n\n".join([doc.page_content for doc in results])

# Print the context to verify
print("Context:")
print(context)

AttributeError: 'list' object has no attribute 'page_content'

In [150]:
# Combine default template and template end
template = default_template() + template_end()

# Create a prompt template for the final response
final_prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

# Create a chain to generate the final response
response_chain = LLMChain(
    llm=llm,
    prompt=final_prompt,
    )

# Generate the final answer using the context and query
final_answer = response_chain.run({"context": context, "question": query})

# Print the final answer
print(f'context: {context}')

print("Final Answer:")
print(final_answer)


context: de Tendre, we mapped our EER projects and theirCopenhagen, DKBack'The Journeys and Territories ofEER' – A Two-Day WorkshopEER | 'The Journeys and Territories of EER' – A Two-Day Workshophttps://www.eer.info/events/the-journeys-and-territories

Okay, I just wanted to say that, I think that EER, as a project, of course, it's built into it is also this thing about iterations, right? Looking at and reflecting, right, experiencing experimenting with something people haven't experienced, we haven't experienced, there's a reflection on it. And then we keep going. And so some of these projects have had many iterations and we used to is one of them that has had quite a few that we can track and then we can look at where were their dead ends,

WEBSITEEditorial: Sophie Erlund & Cordula VesperDesign: Alan WooDevelopment: Daniel MasseyBackAboutEEREER | Abouthttps://www.eer.info/about
5 of 528/01/2024, 18:06

Can you ask it? What kind of aesthetics does EER approach compared with other appr

## trying part 2

In [163]:
# Initialize HuggingFace Embeddings
embeddings = HuggingFaceEmbeddings()

# Initialize Pinecone
pinecone_instance = pc(api_key=os.getenv('PINECONE_API_KEY'), embeddings=embeddings)

# Connect to an existing Pinecone index
index_name = "boteer"
docsearch = Pinecone.from_existing_index(index_name, embeddings)

In [158]:
# Create a MultiQueryRetriever
prompt = PromptTemplate(input_variables=["question"], template="Generate multiple search queries for the following question: {question}")
output_parser = LineListOutputParser()

# Create the query generation chain using RunnableSequence
llm_chain = prompt | llm | output_parser

retriever = MultiQueryRetriever.from_llm(
    retriever=docsearch.as_retriever(),
    llm=llm,
    include_original=True
)

In [160]:
from langchain.callbacks.base import BaseCallbackManager

# Define default templates
def default_template() -> str:
    return """
    You are a chatbot working for the Experimenting Experiencing Reflecting (EER) Project, a research endeavor investigating the connections between art and science.
    You have access to a collection of documents, including descriptions of research activities, meeting transcripts, and other relevant materials.
    All questions should pertain to the EER Project unless specified otherwise.
    """

def template_end() -> str:
    return """
    Context: {context}
    Question: {question}
    Answer: 
    """

# Create a minimal callback manager
class DummyCallbackManager(BaseCallbackManager):
    def get_child(self):
        return self

callback_manager = DummyCallbackManager(handlers=[], inheritable_handlers=[])


In [164]:
retriever.get_relevant_documents(query)

[Document(page_content='most updated version.', metadata={'date_time': 'Thu Apr 20 00:13:31 2023 ', 'source': '2023-04-20 08.16.21_rf.csv', 'speaker_name': 'Peter Dalsgaard'}),
 Document(page_content="So there's two what two versions, this was the first one then this is sort of", metadata={'date_time': 'Thu Mar 16 00:27:38 2023 ', 'source': '2023-03-16 09.01.59_rf.csv', 'speaker_name': 'Helene Nymann'}),
 Document(page_content="to figure out, how do we guide people through these different versions, because in the end, they'll probably be something like five or six different tasks, you can try and you can go back to the menu, try a different one, you could do just one, or you can do all five, and each of them in the back. And we'll create those transmission chains. So I think it's on a good path. And we have all of us attention very much on it. So that's always nice.", metadata={'date_time': 'Fri Jun 18 00:06:50 2021 ', 'source': '2021-06-18 09.01.56_rf.csv', 'speaker_name': 'Sophie Erl

In [176]:
# Define a query
query = "Tell me about the EER project."

# Retrieve documents
results = retriever.batch(query)


AttributeError: 'list' object has no attribute 'metadata'

In [178]:
results

[[Document(page_content='existence of AIS.', metadata={'date_time': 'Thu Mar 16 00:03:20 2023 ', 'source': '2023-03-16 09.01.59_rf.csv', 'speaker_name': 'Peter Dalsgaard'}),
  Document(page_content="this? Incidentally, I'm at an AI workshop AI and humanities workshop organized by Mads Rosendal all of Monday. So as I don't remember the schedule, but if I can join the date, I will great Yeah. Please", metadata={'date_time': 'Fri Dec 16 00:21:51 2022 ', 'source': '2022-12-16 08.40.00_rf.csv', 'speaker_name': 'Peter Dalsgaard'}),
  Document(page_content='a pretty well known thing. But what what I think is also really interesting here is that in many cases, we also see people, and maybe this is particular, or particularly salient for for AI is that people seem to ascribe meaning and agency to the machine. I mean, we even talk about our computer behaving in a stupid way. And we get angry at it as if it was a petulant child or something like that. And I think that maybe there could be somethi

In [181]:
type(results)

list

In [179]:
for doc in results:
    print(doc.page_content)

AttributeError: 'list' object has no attribute 'page_content'

In [177]:
# Combine documents content into a single context with sources
context = ""
for doc in results:
    source = doc.get('source', 'Unknown')
    speaker = doc.get('speaker_name', 'Unknown')
    context += f"Source: {source}\nSpeaker {speaker}\nContent: {doc.page_content}\n\n"

# Print the context to verify
print("Context:")
print(context)

AttributeError: 'list' object has no attribute 'get'

In [162]:
from langchain.prompts import PromptTemplate

# Combine default template and template end
template = default_template() + template_end()

# Create a prompt template for the final response
final_prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

# Create the final response chain using the | operator
response_chain = final_prompt | llm

# Generate the final answer using the context and query
final_answer = response_chain.invoke({"context": context, "question": query})

# Print the final answer
print("Final Answer:")
print(final_answer)


Final Answer:

    The EER (Experimenting Experiencing Reflecting) project is an interdisciplinary research initiative that explores the connections between art and science. The project has gone through several iterations and has been the subject of various workshops, talks, and research activities. The team behind the project consists of researchers with diverse backgrounds, including sociology, visual sociology, and web development. They work together to map out the journeys and territories of the EER projects and discuss interdisciplinarity and its connection to their framework text. The project values reflection and iteration, and the team is always looking for ways to improve and build upon their past work. They have held workshops such as 'The Journeys and Territories of EER' and 'EER and the More-than-human', which bring together various stakeholders to discuss the project's progress and future directions. The project's website, developed by Daniel Massey, provides more informat

In [174]:
from main import ChatBot

In [175]:
bot = ChatBot()

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\szh\.cache\huggingface\token
Login successful


KeyError: 'model'